In [2]:
import matplotlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
import GridWorld_env
from replay_buffer import ReplayBuffer
import gymnasium as gym
import random
import math
from itertools import count

device = "cuda" if torch.cuda.is_available() else "cpu"
is_ipython = "inline" in matplotlib.get_backend()
if is_ipython:
    from IPython import display
    
from collections import deque, namedtuple

Transition = namedtuple("Transition", ("state", "action", "next_state", "reward"))

In [ ]:
class DQN(nn.Module):
    def __init__(self, input_dim, action_dim):
        super(DQN, self).__init__()
               
        self.conv1 = nn.Conv3d(3, 13, 3, 1, 1)
        
        self.conv2 = nn.Conv3d(32, 32, 3, 1, 1)
        
        self.fc1 = nn.Linear((64 + 3*2)*input_dim * input_dim * input_dim, 512)
        
        self.actions = nn.Linear(1024, action_dim)
        self.advantage = nn.Linear(1024, 1)
    def forward(self, x):
        original_state = x
               
        x = self.conv1(x)
        
        x = torch.cat([x, original_state], dim=1)
        x = torch.cat([x, -x], dim = 1)
        x = F.relu(x)
        
        x = self.conv2(x)
        x = torch.cat([x, original_state], dim=1)
        x = torch.cat([x, -x], dim = 1)
        
        x = F.relu(x)
        x = nn.Flatten()(x)
        x = self.fc1(x)
        x = torch.cat([x, -x], dim = -1)
        x = F.relu(x)
        actions = self.actions(x)
        advantage = self.advantage(x)
        x = advantage + (actions - actions.mean(dim=1, keepdim=True))
        return x
"""
test = DQN(4, 7)
import torchinfo
print(torchinfo.summary(test, (1, 3, 4, 4, 4)))"""
